# Industry 4.0 의 중심, AI - ML&DL

<div align='right'><font size=2 color='gray'>Machine Learning & Deep Learning with TensorFlow @ <font color='blue'><a href='https://www.facebook.com/jskim.kr'>FB / jskim.kr</a></font>, 김진수</font></div>
<hr>

# Sect12. RNN (Recurrent Newural Network)

In [1]:
from images import bigpycraft_ai as bpc
from IPython.display import Image 
from tqdm import tqdm_notebook

In [2]:
import time
import os

def chk_processting_time(start_time, end_time):
    process_time = end_time - start_time
    p_time = int(process_time)
    p_min = p_time // 60
    p_sec = p_time %  60
    print('처리시간 : {p_min}분 {p_sec}초 경과되었습니다.'.format(
            p_min = p_min, 
            p_sec = p_sec
        ))
    return process_time

### 12-3. only_softmax_char_seq

<hr>
``` python 

# Lab 12 Character Sequence Softmax only
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility

sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex

# hyper parameters
dic_size = len(char2idx)  # RNN input size (one hot size)
rnn_hidden_size = len(char2idx)  # RNN output size
num_classes = len(char2idx)  # final output size (RNN or softmax, etc.)
batch_size = 1  # one sample data, one batch
sequence_length = len(sample) - 1  # number of lstm rollings (unit #)

sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

# flatten the data (ignore batches for now). No effect if the batch size is 1
X_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
X_for_softmax = tf.reshape(X_one_hot, [-1, rnn_hidden_size])

# softmax layer (rnn_hidden_size -> num_classes)
softmax_w = tf.get_variable("softmax_w", [rnn_hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

# expend the data (revive the batches)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])

# Compute sequence cost/loss
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)  # mean all sequence loss
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t_cnt in range(3000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(t_cnt, "loss:", l, "Prediction:", ''.join(result_str))

'''
 1th training... 	 Loss : 2.599400520324707, 	 Prediction : uyuunuiutnffunu 
 2th training... 	 Loss : 2.378079414367676, 	 Prediction : uyuunu utnfuunu 
 3th training... 	 Loss : 2.1724300384521484, 	 Prediction : uyuuuu utntuuuu 
 4th training... 	 Loss : 1.980708122253418, 	 Prediction : iy iou iynt iou 
 5th training... 	 Loss : 1.8024282455444336, 	 Prediction : yy you yynt you 

..
..

 2995th training... 	 Loss : 0.2773257791996002, 	 Prediction : yf you yant you 
 2996th training... 	 Loss : 0.2773257791996002, 	 Prediction : yf you yant you 
 2997th training... 	 Loss : 0.2773257791996002, 	 Prediction : yf you yant you 
 2998th training... 	 Loss : 0.27732571959495544, 	 Prediction : yf you yant you 
 2999th training... 	 Loss : 0.27732568979263306, 	 Prediction : yf you yant you 
 3000th training... 	 Loss : 0.2773256301879883, 	 Prediction : yf you yant you 
 
'''


```

<hr>

In [3]:
# Lab 12 Character Sequence Softmax only
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # reproducibility


C:\Python\Anaconda3-50\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
sample = " if you want you"
idx2char = list(set(sample))  # index -> char
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex


In [5]:
# hyper parameters
dic_size        = len(char2idx)     # RNN input size (one hot size)
rnn_hidden_size = len(char2idx)     # RNN output size
num_classes     = len(char2idx)     # final output size (RNN or softmax, etc.)
batch_size      = 1                 # one sample data, one batch
sequence_length = len(sample) - 1   # number of lstm rollings (unit #)

In [6]:
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]    # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]     # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label


In [7]:
# flatten the data (ignore batches for now). No effect if the batch size is 1
X_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
X_for_softmax = tf.reshape(X_one_hot, [-1, rnn_hidden_size])

# softmax layer (rnn_hidden_size -> num_classes)
softmax_w = tf.get_variable("softmax_w", [rnn_hidden_size, num_classes])
softmax_b = tf.get_variable("softmax_b", [num_classes])
outputs = tf.matmul(X_for_softmax, softmax_w) + softmax_b

# expend the data (revive the batches)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])

# Compute sequence cost/loss
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)  # mean all sequence loss
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [8]:
time1 = time.time()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for t_cnt in range(3000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})

        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]

        # print(t_cnt, "loss:", l, "Prediction:", ''.join(result_str))
        print(" {cnt}th training... \t Loss : {loss}, \t Prediction : {pred} ".format(
                cnt  = t_cnt+1, 
                loss = l, 
                pred = ''.join(result_str)
        ))

time2 = time.time()

 1th training... 	 Loss : 2.599400520324707, 	 Prediction : uyuunuiutnffunu 
 2th training... 	 Loss : 2.378079414367676, 	 Prediction : uyuunu utnfuunu 
 3th training... 	 Loss : 2.1724300384521484, 	 Prediction : uyuuuu utntuuuu 
 4th training... 	 Loss : 1.980708122253418, 	 Prediction : iy iou iynt iou 
 5th training... 	 Loss : 1.8024282455444336, 	 Prediction : yy you yynt you 
 6th training... 	 Loss : 1.6376303434371948, 	 Prediction : yy you yynt you 
 7th training... 	 Loss : 1.4861983060836792, 	 Prediction : yy you yant you 
 8th training... 	 Loss : 1.3480725288391113, 	 Prediction : yy you yant you 
 9th training... 	 Loss : 1.2229617834091187, 	 Prediction : yy you yant you 
 10th training... 	 Loss : 1.1102442741394043, 	 Prediction : yf you yant you 
 11th training... 	 Loss : 1.0091509819030762, 	 Prediction : yf you yant you 
 12th training... 	 Loss : 0.9189122915267944, 	 Prediction : yf you yant you 
 13th training... 	 Loss : 0.8387787342071533, 	 Prediction : yf

 104th training... 	 Loss : 0.2884511351585388, 	 Prediction : yf you yant you 
 105th training... 	 Loss : 0.28833889961242676, 	 Prediction : yf you yant you 
 106th training... 	 Loss : 0.28822848200798035, 	 Prediction : yf you yant you 
 107th training... 	 Loss : 0.28812000155448914, 	 Prediction : yf you yant you 
 108th training... 	 Loss : 0.28801319003105164, 	 Prediction : yf you yant you 
 109th training... 	 Loss : 0.2879081070423126, 	 Prediction : yf you yant you 
 110th training... 	 Loss : 0.2878046929836273, 	 Prediction : yf you yant you 
 111th training... 	 Loss : 0.28770291805267334, 	 Prediction : yf you yant you 
 112th training... 	 Loss : 0.2876027524471283, 	 Prediction : yf you yant you 
 113th training... 	 Loss : 0.2875041365623474, 	 Prediction : yf you yant you 
 114th training... 	 Loss : 0.2874070703983307, 	 Prediction : yf you yant you 
 115th training... 	 Loss : 0.2873114049434662, 	 Prediction : yf you yant you 
 116th training... 	 Loss : 0.28721

 205th training... 	 Loss : 0.28231626749038696, 	 Prediction : yf you yant you 
 206th training... 	 Loss : 0.28228482604026794, 	 Prediction : yf you yant you 
 207th training... 	 Loss : 0.28225359320640564, 	 Prediction : yf you yant you 
 208th training... 	 Loss : 0.2822226881980896, 	 Prediction : yf you yant you 
 209th training... 	 Loss : 0.2821921706199646, 	 Prediction : yf you yant you 
 210th training... 	 Loss : 0.2821619510650635, 	 Prediction : yf you yant you 
 211th training... 	 Loss : 0.28213196992874146, 	 Prediction : yf you yant you 
 212th training... 	 Loss : 0.2821023464202881, 	 Prediction : yf you yant you 
 213th training... 	 Loss : 0.28207290172576904, 	 Prediction : yf you yant you 
 214th training... 	 Loss : 0.28204381465911865, 	 Prediction : yf you yant you 
 215th training... 	 Loss : 0.28201499581336975, 	 Prediction : yf you yant you 
 216th training... 	 Loss : 0.2819865345954895, 	 Prediction : yf you yant you 
 217th training... 	 Loss : 0.281

 306th training... 	 Loss : 0.2802213430404663, 	 Prediction : yf you yant you 
 307th training... 	 Loss : 0.2802080512046814, 	 Prediction : yf you yant you 
 308th training... 	 Loss : 0.2801946997642517, 	 Prediction : yf you yant you 
 309th training... 	 Loss : 0.2801816463470459, 	 Prediction : yf you yant you 
 310th training... 	 Loss : 0.2801685333251953, 	 Prediction : yf you yant you 
 311th training... 	 Loss : 0.28015556931495667, 	 Prediction : yf you yant you 
 312th training... 	 Loss : 0.2801426947116852, 	 Prediction : yf you yant you 
 313th training... 	 Loss : 0.28012987971305847, 	 Prediction : yf you yant you 
 314th training... 	 Loss : 0.2801172435283661, 	 Prediction : yf you yant you 
 315th training... 	 Loss : 0.2801046073436737, 	 Prediction : yf you yant you 
 316th training... 	 Loss : 0.28009212017059326, 	 Prediction : yf you yant you 
 317th training... 	 Loss : 0.28007972240448, 	 Prediction : yf you yant you 
 318th training... 	 Loss : 0.280067324

 407th training... 	 Loss : 0.27923819422721863, 	 Prediction : yf you yant you 
 408th training... 	 Loss : 0.2792312204837799, 	 Prediction : yf you yant you 
 409th training... 	 Loss : 0.2792242169380188, 	 Prediction : yf you yant you 
 410th training... 	 Loss : 0.27921730279922485, 	 Prediction : yf you yant you 
 411th training... 	 Loss : 0.2792103886604309, 	 Prediction : yf you yant you 
 412th training... 	 Loss : 0.2792036235332489, 	 Prediction : yf you yant you 
 413th training... 	 Loss : 0.2791967988014221, 	 Prediction : yf you yant you 
 414th training... 	 Loss : 0.27918997406959534, 	 Prediction : yf you yant you 
 415th training... 	 Loss : 0.2791832387447357, 	 Prediction : yf you yant you 
 416th training... 	 Loss : 0.2791765332221985, 	 Prediction : yf you yant you 
 417th training... 	 Loss : 0.27916982769966125, 	 Prediction : yf you yant you 
 418th training... 	 Loss : 0.27916327118873596, 	 Prediction : yf you yant you 
 419th training... 	 Loss : 0.27915

...

 2513th training... 	 Loss : 0.2773553133010864, 	 Prediction : yf you yant you 
 2514th training... 	 Loss : 0.27735528349876404, 	 Prediction : yf you yant you 
 2515th training... 	 Loss : 0.2773551344871521, 	 Prediction : yf you yant you 
 2516th training... 	 Loss : 0.27735504508018494, 	 Prediction : yf you yant you 
 2517th training... 	 Loss : 0.27735501527786255, 	 Prediction : yf you yant you 
 2518th training... 	 Loss : 0.277354896068573, 	 Prediction : yf you yant you 
 2519th training... 	 Loss : 0.277354896068573, 	 Prediction : yf you yant you 
 2520th training... 	 Loss : 0.27735480666160583, 	 Prediction : yf you yant you 
 2521th training... 	 Loss : 0.2773546874523163, 	 Prediction : yf you yant you 
 2522th training... 	 Loss : 0.2773546278476715, 	 Prediction : yf you yant you 
 2523th training... 	 Loss : 0.27735456824302673, 	 Prediction : yf you yant you 
 2524th training... 	 Loss : 0.27735447883605957, 	 Prediction : yf you yant you 
 2525th training... 	 Lo

 2613th training... 	 Loss : 0.27734801173210144, 	 Prediction : yf you yant you 
 2614th training... 	 Loss : 0.27734795212745667, 	 Prediction : yf you yant you 
 2615th training... 	 Loss : 0.2773478627204895, 	 Prediction : yf you yant you 
 2616th training... 	 Loss : 0.2773478031158447, 	 Prediction : yf you yant you 
 2617th training... 	 Loss : 0.2773476541042328, 	 Prediction : yf you yant you 
 2618th training... 	 Loss : 0.2773475646972656, 	 Prediction : yf you yant you 
 2619th training... 	 Loss : 0.27734753489494324, 	 Prediction : yf you yant you 
 2620th training... 	 Loss : 0.27734747529029846, 	 Prediction : yf you yant you 
 2621th training... 	 Loss : 0.2773474454879761, 	 Prediction : yf you yant you 
 2622th training... 	 Loss : 0.2773473262786865, 	 Prediction : yf you yant you 
 2623th training... 	 Loss : 0.27734729647636414, 	 Prediction : yf you yant you 
 2624th training... 	 Loss : 0.277347207069397, 	 Prediction : yf you yant you 
 2625th training... 	 Lo

 2713th training... 	 Loss : 0.2773413360118866, 	 Prediction : yf you yant you 
 2714th training... 	 Loss : 0.2773412764072418, 	 Prediction : yf you yant you 
 2715th training... 	 Loss : 0.27734124660491943, 	 Prediction : yf you yant you 
 2716th training... 	 Loss : 0.27734115719795227, 	 Prediction : yf you yant you 
 2717th training... 	 Loss : 0.2773410975933075, 	 Prediction : yf you yant you 
 2718th training... 	 Loss : 0.2773410677909851, 	 Prediction : yf you yant you 
 2719th training... 	 Loss : 0.27734097838401794, 	 Prediction : yf you yant you 
 2720th training... 	 Loss : 0.2773408889770508, 	 Prediction : yf you yant you 
 2721th training... 	 Loss : 0.277340829372406, 	 Prediction : yf you yant you 
 2722th training... 	 Loss : 0.27734076976776123, 	 Prediction : yf you yant you 
 2723th training... 	 Loss : 0.27734076976776123, 	 Prediction : yf you yant you 
 2724th training... 	 Loss : 0.27734071016311646, 	 Prediction : yf you yant you 
 2725th training... 	 L

 2813th training... 	 Loss : 0.27733540534973145, 	 Prediction : yf you yant you 
 2814th training... 	 Loss : 0.27733534574508667, 	 Prediction : yf you yant you 
 2815th training... 	 Loss : 0.2773352861404419, 	 Prediction : yf you yant you 
 2816th training... 	 Loss : 0.2773352563381195, 	 Prediction : yf you yant you 
 2817th training... 	 Loss : 0.27733510732650757, 	 Prediction : yf you yant you 
 2818th training... 	 Loss : 0.27733510732650757, 	 Prediction : yf you yant you 
 2819th training... 	 Loss : 0.2773350179195404, 	 Prediction : yf you yant you 
 2820th training... 	 Loss : 0.277334988117218, 	 Prediction : yf you yant you 
 2821th training... 	 Loss : 0.27733489871025085, 	 Prediction : yf you yant you 
 2822th training... 	 Loss : 0.27733486890792847, 	 Prediction : yf you yant you 
 2823th training... 	 Loss : 0.2773348391056061, 	 Prediction : yf you yant you 
 2824th training... 	 Loss : 0.2773347496986389, 	 Prediction : yf you yant you 
 2825th training... 	 L

 2913th training... 	 Loss : 0.2773299813270569, 	 Prediction : yf you yant you 
 2914th training... 	 Loss : 0.2773299813270569, 	 Prediction : yf you yant you 
 2915th training... 	 Loss : 0.2773298919200897, 	 Prediction : yf you yant you 
 2916th training... 	 Loss : 0.2773298919200897, 	 Prediction : yf you yant you 
 2917th training... 	 Loss : 0.27732983231544495, 	 Prediction : yf you yant you 
 2918th training... 	 Loss : 0.2773298919200897, 	 Prediction : yf you yant you 
 2919th training... 	 Loss : 0.27733004093170166, 	 Prediction : yf you yant you 
 2920th training... 	 Loss : 0.2773301899433136, 	 Prediction : yf you yant you 
 2921th training... 	 Loss : 0.27733054757118225, 	 Prediction : yf you yant you 
 2922th training... 	 Loss : 0.27733108401298523, 	 Prediction : yf you yant you 
 2923th training... 	 Loss : 0.277332067489624, 	 Prediction : yf you yant you 
 2924th training... 	 Loss : 0.27733343839645386, 	 Prediction : yf you yant you 
 2925th training... 	 Lo

In [9]:
chk_processting_time(time1, time2)

처리시간 : 0분 11초 경과되었습니다.


11.433363676071167

<hr>
<marquee><font size=3 color='brown'>The BigpyCraft find the information to design valuable society with Technology & Craft.</font></marquee>
<div align='right'><font size=2 color='gray'> &lt; The End &gt; </font></div>